In [1]:
import sys
from pathlib import Path

import numpy as np
import pandas as pd

sys.path.append(str(Path.resolve(Path.cwd() / ".." / "..")))

import new_util as util

DATA_DIR = "../../../temp"
MONTHS, YEARS = util.get_months_years(DATA_DIR)
TIME_DICT = util.get_month_year_dicts(DATA_DIR)

# TODO: delete files in data folder and delete content of plots.json?
def reset_data_and_plots():
    pass

reset_data_and_plots()

In [9]:
def get_top_k_contributor_count(ddf, tag, k):
    if tag in ["created_by", "corporation", "streetcomplete"]:
        highest_number = np.iinfo(ddf[tag].dtype).max
        ddf = ddf[ddf[tag] < highest_number]

    # counting the contibutor count only for tags that have a minimum amount of changesets for efficiency
    # min_amount_of_changeset should be lower then the contributor count of the top 500 from previous runs
    total_changeset_count = ddf.groupby(tag).size().compute().sort_values(ascending=False)
    tag_to_min_amount_of_changeset = {
        "created_by": 30,
        "imagery": 500,
        "hashtag": 2500,
        "source": 250,
        "corporation": 0,
        "streetcomplete": 0,
    }
    min_amount_of_changeset = tag_to_min_amount_of_changeset.get(tag, 0)
    filtered_ddf = ddf[ddf[tag].isin(total_changeset_count[total_changeset_count > min_amount_of_changeset].index)]

    total_contributor_count = filtered_ddf.groupby(tag)["user_index"].nunique().compute().sort_values(ascending=False)

    indices = total_contributor_count.index.to_numpy()[:k]
    index_to_tag = util.load_index_to_tag(DATA_DIR, tag)
    names = [index_to_tag[i] for i in indices]
    
    values = total_contributor_count[indices[:k]].to_numpy()
    return indices[:k], names[:k], values

def get_top_k_edit_count(ddf, tag, k):
    if tag in ["created_by", "corporation", "streetcomplete"]:
        highest_number = np.iinfo(ddf[tag].dtype).max
        ddf = ddf[ddf[tag] < highest_number]
    
    total_edit_count = ddf.groupby(tag)["edits"].sum().compute().sort_values(ascending=False)

    indices = total_edit_count.index.to_numpy()[:k]
    index_to_tag = util.load_index_to_tag(DATA_DIR, tag)
    names = [index_to_tag[i] for i in indices]
    
    values = total_edit_count[indices[:k]].to_numpy()
    return indices[:k], names[:k], values

def get_top_k_changeset_count(ddf, tag, k):
    if tag in ["created_by", "corporation", "streetcomplete"]:
        highest_number = np.iinfo(ddf[tag].dtype).max
        ddf = ddf[ddf[tag] < highest_number]
    
    total_changeset_count = ddf.groupby(tag).size().compute().sort_values(ascending=False)

    indices = total_changeset_count.index.to_numpy()[:k]
    index_to_tag = util.load_index_to_tag(DATA_DIR, tag)
    names = [index_to_tag[i] for i in indices]

    values = total_changeset_count[indices[:k]].to_numpy()
    return indices[:k], names[:k], values


def top_k_unique_contributor_monthly_list(contributor_count_ddf_filename, tag, indices):
    unique_monthly_list = [[set() for _ in range(len(MONTHS))] for _ in range(len(indices))]

    for month_i in range(len(MONTHS)):
        filters = [("month_index", "==", month_i)]
        temp_ddf = util.load_ddf(DATA_DIR, contributor_count_ddf_filename, ("user_index", tag), filters)
        contributor_unique_month = (
            temp_ddf[temp_ddf[tag].isin(indices)].groupby([tag], observed=False)["user_index"].unique().compute()
        )
        for i, tag_index in enumerate(indices):
            value = contributor_unique_month[contributor_unique_month.index == tag_index].to_numpy()
            if len(value) > 0:
                unique_monthly_list[i][month_i] = set(value[0].tolist())
    return unique_monthly_list

# Contributor Count

In [10]:
ddf = util.load_ddf(DATA_DIR, "general", ["month_index", "user_index", "created_by"])
k = 100
top_100_contrib_indices, top_100_contrib_names, top_100_contrib_values = get_top_k_contributor_count(ddf, "created_by", k)
unique_monthly_list = top_k_unique_contributor_monthly_list("general", "created_by", top_100_contrib_indices)

In [11]:
top_100_monthly_contributor_count = pd.DataFrame(np.array([[len(unique_set) for unique_set in month_list] for month_list in unique_monthly_list]).transpose(), columns=top_100_contrib_names)
top_100_monthly_contributor_count = top_100_monthly_contributor_count.rename_axis("month_index").reset_index()
util.save_data(DATA_DIR, "created_by_top_100_contributor_count_monthly", top_100_monthly_contributor_count)
util.save_data(DATA_DIR, "created_by_top_10_contributor_count_monthly", top_100_monthly_contributor_count.iloc[:,:10+1])

In [12]:
plot_config = util.get_plot_config("created_by_top_10_contributor_count_monthly", "monthly contributors per editing software", y_unit="contributors")
util.save_plot_config(plot_config)
util.show_plot(plot_config)

In [13]:
year_index_to_month_indices = util.get_year_index_to_month_indices(DATA_DIR)

contributor_count_yearly_list = []
for month_list in unique_monthly_list:
    contributor_count_yearly = []
    for year_i, _ in enumerate(YEARS):
        contributor_count_yearly.append(
            len(set().union(*[month_list[month_i] for month_i in year_index_to_month_indices[year_i]])),
        )
    contributor_count_yearly_list.append(contributor_count_yearly)

top_100_yearly_contributor_count = pd.DataFrame(np.array(contributor_count_yearly_list).transpose(), columns=top_100_contrib_names).rename_axis("year_index").reset_index()
top_100_yearly_contributor_count_and_total = top_100_yearly_contributor_count
util.save_data(DATA_DIR, "created_by_top_100_contributor_count_yearly", top_100_yearly_contributor_count)

In [15]:
plot_config = util.get_html_table_str("created_by_top_100_contributor_count_yearly", transpose_new_index_column="years", add_rank=True, last_column_name_values=("total", top_100_contrib_values))
util.save_plot_config(plot_config)
util.show_plot(plot_config)

Rank,Editor,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,total
1,iD,0,0,0,0,"56,176","125,149","133,826","148,447","194,537","214,057","203,438","241,197","239,537","204,579","210,136","29,969","1,413,350"
2,Potlatch,"59,540","69,047","81,896","107,158","72,954","29,604","24,286","14,901","10,184","6,598","5,043","3,531",608,350,234,77,"358,269"
3,MAPS.ME android,0,0,0,0,0,0,0,"60,058","63,947","37,666","32,622","20,331","12,043","6,626","10,651","1,279","192,520"
4,JOSM,"13,755","18,797","20,232","23,441","23,140","21,862","22,796","22,315","23,191","22,762","22,801","22,109","21,512","20,536","19,741","5,914","143,216"
5,MAPS.ME ios,0,0,0,0,0,0,0,"37,424","39,443","34,149","23,541","15,385","8,052","8,966","5,658",453,"129,302"
6,StreetComplete,0,0,0,0,0,0,0,7,"9,572","9,764","8,967","11,695","21,935","23,525","26,809","5,404","65,258"
7,OsmAnd,0,192,645,"1,129","1,653","1,927","2,354","3,534","4,863","5,982","7,064","6,978","7,959","8,623","8,544","1,424","33,616"
8,Vespucci,58,237,460,957,"1,622","1,801","2,075","2,380","2,920","3,332","3,632","4,314","4,539","5,025","5,371","1,044","21,349"
9,Go Map!!,0,0,0,0,"2,205","2,046","1,335","1,508","1,834","2,685","3,074","3,152","3,748","3,720","3,830",873,"16,716"
10,Rapid,0,0,0,0,0,0,0,0,0,0,"1,105","3,709","2,124","2,183","9,121",537,"15,683"


In [16]:
ddf = util.load_ddf(DATA_DIR, "general", ["month_index", "user_index"])
contributors_unique_monthly = ddf.groupby(["month_index"], observed=False)["user_index"].unique().rename("contributors").compute()
util.save_data(DATA_DIR, "general_contributor_count_monthly", contributors_unique_monthly.apply(len))

util.save_percent(
    "created_by_top_10_contributor_count_monthly",
    "general_contributor_count_monthly",
    "months",
    "contributors",
)

In [17]:
plot_config = util.get_plot_config("created_by_top_10_contributor_count_monthly_percent", "percent of contributors that use each editing software per month", y_unit="%")
util.save_plot_config(plot_config)
util.show_plot(plot_config)

## monthly first editing software contributor count

In [48]:
ddf = util.load_ddf(DATA_DIR, "general", ["month_index", "user_index", "created_by"])
tag = "created_by"
first_edit_per_user = (ddf.groupby(["user_index"], observed=False)["month_index", tag].first().compute())
first_contibutor_count_monthly = (first_edit_per_user.reset_index().groupby(["month_index", tag], observed=False)["user_index"].count())
first_contibutor_count_monthly = first_contibutor_count_monthly.rename("contributors").reset_index()

first_contibutor_count_monthly = pd.pivot_table(first_contibutor_count_monthly, values="contributors", index="month_index", columns=tag)[top_100_contrib_indices[:10]]
first_contibutor_count_monthly.columns = top_100_contrib_names[:10]
first_contibutor_count_monthly = first_contibutor_count_monthly.reset_index()
util.save_data(DATA_DIR, "created_by_top_10_contributor_count_first_changeset_monthly", first_contibutor_count_monthly)

In [51]:
util.save_data(DATA_DIR, "general_new_contributor_count_monthly", util.cumsum_new_nunique(contributors_unique_monthly))
util.save_percent(
    "created_by_top_10_contributor_count_first_changeset_monthly",
    "general_new_contributor_count_monthly",
    "months",
    "contributors",
)

In [53]:
plot_config = util.get_plot_config("created_by_top_10_contributor_count_first_changeset_monthly_percent", "first editor of new contributors per month", y_unit="%")
util.save_plot_config(plot_config)
util.show_plot(plot_config)

# Edit Count

In [18]:
ddf = util.load_ddf(DATA_DIR, "general", ["month_index", "edits", "created_by"])
k = 100
top_100_edit_indices, top_100_edit_names, top_100_edit_values = get_top_k_edit_count(ddf, "created_by", k)

In [19]:
monthly_edit_count_df = (
    ddf[ddf["created_by"].isin(top_100_edit_indices)]
    .groupby(["month_index", "created_by"], observed=False)["edits"]
    .sum()
    .compute()
    .reset_index()
)
monthly_edit_count_df = pd.pivot_table(monthly_edit_count_df, values="edits", index="month_index", columns="created_by")[top_100_edit_indices]
monthly_edit_count_df.columns = top_100_edit_names
monthly_edit_count_df = monthly_edit_count_df.reset_index()
util.save_data(DATA_DIR, "created_by_top_100_edit_count_monthly", monthly_edit_count_df)
util.save_data(DATA_DIR, "created_by_top_10_edit_count_monthly", monthly_edit_count_df.iloc[:, :10+1])

In [20]:
plot_config = util.get_plot_config("created_by_top_10_edit_count_monthly", "monthly edits per editing software", y_unit="edits")
util.save_plot_config(plot_config)
util.show_plot(plot_config)

In [23]:
util.save_monthly_to_yearly("created_by_top_100_edit_count_monthly")
plot_config = util.get_html_table_str("created_by_top_100_edit_count_yearly", transpose_new_index_column="years", add_rank=True, last_column_name_values=("total", top_100_edit_values))
util.save_plot_config(plot_config)
util.show_plot(plot_config)

Rank,Editor,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,total
1,JOSM,"125,617,700","494,460,089","539,095,272","542,112,695","500,031,282","666,695,875","642,724,561","625,396,093","663,901,878","793,994,770","790,692,615","996,035,337","917,479,929","758,553,217","756,755,624","85,510,186","9,899,057,123"
2,iD,0,0,0,0,"34,430,862","109,275,830","142,202,485","189,396,418","279,162,048","321,342,148","359,664,267","459,865,024","480,370,002","416,745,824","427,963,559","39,107,936","3,259,526,403"
3,Potlatch,"44,030,413","64,726,838","100,975,427","147,722,227","133,843,984","78,110,810","60,205,870","43,031,441","33,578,906","25,192,118","20,654,187","17,405,786","5,083,884","2,678,828","2,574,667","220,722","780,036,108"
4,Rapid,0,0,0,0,0,0,0,0,0,0,"11,678,103","51,870,526","63,919,308","63,564,994","63,683,830","4,745,709","259,462,470"
5,osmtools,"150,412,240","8,994,328","981,921","1,229,689","2,734,257","1,499,370","6,826,400","15,035,558","2,657,362","3,272,976","7,558,685","6,454,470","8,152,531","12,399,003","29,128,241","45,092","257,382,123"
6,bulk_upload.py,"64,669,041","24,649,625","1,442,184","759,090","185,033","8,089","1,061,001","1,113,638","2,398","1,036","26,933,301","263,564","464,798",0,0,0,"121,552,798"
7,Merkaartor,"5,378,663","10,887,187","10,250,061","9,702,870","4,380,406","3,791,724","3,696,670","2,523,573","1,735,352","2,138,115","2,316,287","1,745,000","334,114","491,775","123,828","4,594","59,500,219"
8,upload.py,"31,167,015","10,166,002","2,424,682","421,238","776,590","422,731","488,194","4,827","38,997","3,021","4,195","1,853,644","4,378,760","822,536","968,637","2,739","53,943,808"
9,StreetComplete,0,0,0,0,0,0,0,85,"1,256,645","1,701,151","1,983,767","3,934,683","12,440,285","12,905,084","13,630,661","837,371","48,689,732"
10,osmapi,0,0,0,0,0,"797,190","139,521","682,197","288,536","240,521","162,631","2,095,734","23,940,913","1,860,602","1,030,376","23,192","31,261,413"


In [24]:
# Which editor is used for the first edit?
# Ich möchte von jeder Person den ersten Beitrag haben. Aus dem Beitrag möchte ich (Monat, created_by)
# Dann kann ich zählen wie oft was vorkommt und dann habe ich die Tabelle.


#ddf = util.load_ddf(DATA_DIR, "general", ["month_index", "created_by"])
#k = 100
#top_100_change_indices, top_100_change_names, top_100_change_values = get_top_k_changeset_count(ddf, "created_by", k)

In [ ]:


#contibutor_monthly = (ddf[ddf[tag].isin(indices)].groupby(["user_index"], observed=False)["month_index", tag].first().compute())
#contibutor_count_monthly = (contibutor_monthly.reset_index().groupby(["month_index", tag], observed=False)["user_index"].count())

In [ ]:

#ddf": load_parquet_func("general", ["month_index", "user_index", "created_by"]),

def get_tag_top_10_contributor_count_first_changeset_monthly(ddf, tag):
    indices, names = get_cached_top_k(ddf, tag, "contributor_count", 10)

    
    
    contibutor_count_monthly = contibutor_count_monthly.rename("contributors").reset_index()

    df = pd.pivot_table(contibutor_count_monthly, values="contributors", index="month_index", columns=tag)[indices]
    df.columns = names
    return df.reset_index()